## Instance: GPStoActionspace()
### Input
Takes as input the output of `GPSAnalytics().metrics.get_metrics()` -> act (dataframe)

### Introduction
This notebook applies centrography to the MOBIS data at state 2, focusing on activity-level aggregation.

The objective is to generate key metrics characterizing the activity space for a more in-depth exploration of spatial familiarity.

Spatial familiarity metrics encompass a composite evaluation of location history, daily activity-space variability, and spatial innovation. Achieving this involves intricate data transformations utilizing advanced point-pattern centrography. Leveraging a dataset with labeled locations, including purpose and visit counts, over a specific time frame, marked point pattern analysis (PPA) facilitates the study of individual action spaces (Baddeley, Rubak, and Turner 2015).

The implementation of centrography (utilizing the Python Spatial Analysis library) extracts characteristics to describe the activity space:

- **Points**: Marked visited places with counts of visits, purpose labels (home, work, leisure, duties), unique location IDs, and intensity (average number of event points per unit of the convex hull area).
- **Centers**: The mean center and weighted mean centers (weighted by the count of visits).
- **Distances**: Standard distance, offering a one-dimensional measure of how dispersed visited locations are around their mean center, and the sum of distances from home.
- **Shapes**: Standard deviational ellipse, providing a two-dimensional measure of the dispersion of visited locations, and the minimum convex hull of frequently visited places.

This approach predominantly relies on the Python library for spatial analysis, PySAL.

### Public methods
The public methods should be the following:
- `GSPtoActionspace().compute_action_space(act, aggreg_method = 'user_id'/'user_id_day',plot_ellipses = False)` -> AS (dataframe) Get from Part 0 and 1 and 2 below
- `GSPtoActionspace().covariance_matric(AS)` Get from Part 3 below
- `GSPtoActionspace().plot_action_space(act, AS, user_subset = ['CH15029', 'CH16871'], how = 'vignette'/'folium', save = False)` Get from Part 4 below
- `GPStoActionspace().inno_rate(mtf_, AS_day, user_id_, phase=None, treatment=None)` Get from Part 5 below

### Methodology

$$
\text{(Eq. 3) } \quad
Regularity = \dfrac{n_f + 1}{n} \text{; with $n_f$ the number of frequently visited locations and $n$ the total number of locations} 
\\ \text{(Eq. 4) }  \quad
    Frequency=
    \begin{cases}
      \text{'most visited'}, & \text{for}\ \arg\max({f_i}) \\
      \text{'frequent visits'}, & \text{for}\ f_i > 0.5 \cdot \arg\max({f_i}) \\
      \text{'occasional visits'}, & \text{for}\ f_i \leq 0.5 \cdot \arg\max({f_i}) \\
      \text{'visited once'}, & \text{if}\ f_i = 1
    \end{cases} 
    \text{with $f_i$ the count of visits at location $i$}
\\ \text{(Eq. 5) }  \quad
    Proximity =
    \dfrac{SD_{freq}}{SD_{all}} \text{ is }
        \begin{cases}
          > 1 \text{ for dispersed habitual activity space and close innovative activity space} \\
          \approx 1 \text{ for homogeneous activity spaces} \\
          < 1 \text{ for dispersed innovative activity space and close habitual activity space }
        \end{cases} 
\\
\qquad  \text{Given the coordinates $(x,y)$ of $i$ locations :}
\\
\qquad  \text{(5.1) } \text{with}  \quad
SD_{freq} = \displaystyle \sqrt{\frac{\sum^n_{i=1}(x_i-x_{home})^2}{n} + \frac{\sum^n_{i=1}(y_i-y_{home})^2}{n}}
\text{  } \forall \text{nodes } n_i(x_i,y_i)\in C_f = \{\text{'frequently visited places', 'most visited place'}\}
\\ \qquad \text{(5.2) } \text{and}  \quad
SD_{all} = \displaystyle \sqrt{\frac{\sum^n_{i=1}(x_i-x_{home})^2}{n} + \frac{\sum^n_{i=1}(y_i-y_{home})^2}{n}}
\text{  } \forall \text{nodes } n_i(x_i,y_i) \in C =\{\text{'all visited places}\}
\\ \text{(Eq. 6) }  \quad
    \textit{Home shift} = 
    \sqrt {\left( {x_{home} - x_{wmc} } \right)^2 + \left( {y_{home} - y_{wmc} } \right)^2 }
$$

In [ ]:
from functions_action_space import *

# Standard Libraries
import math
from datetime import datetime
import multiprocessing as mp
from pathos.multiprocessing import ProcessingPool as Pool

# Data Manipulation and Analysis
import pandas as pd
import geopandas as gpd
from geopandas import sjoin
import numpy as np
import time
import utm
import pyproj

# Visualization
import plotly.graph_objects as go
import seaborn as sns
sns.set_theme(style="white")
sns.set_palette("Pastel2_r")
cm = sns.light_palette("#fff4ac",as_cmap=True)
from matplotlib import pyplot as plt
import folium
import pyproj

# Point Pattern Analysis and Spatial Statistics
from pointpats.centrography import hull, mbr, mean_center, weighted_mean_center, manhattan_median, std_distance, euclidean_median, ellipse, dtot
from shapely.geometry import MultiPoint

from scipy.spatial import ConvexHull, convex_hull_plot_2d
from haversine import haversine, Unit
from matplotlib.patches import Ellipse
from pylab import figure, show, rand

from pointpats import PointPattern
from scipy.spatial import distance
from pysal.explore.pointpats import mean_center, weighted_mean_center, std_distance, euclidean_median, ellipse
from sklearn.metrics.pairwise import haversine_distances
from geopy.distance import geodesic

import libpysal as ps
from pointpats import PointPattern

plt.rcParams['figure.figsize'] = [5, 5]
plt.rcParams['figure.dpi'] = 120 # 200 e.g. is really fine, but slower

In [ ]:
act = pd.read_pickle('sample_data/extended_staypoint_sample_panel.pkl')
act.head(3)

### Part 0
Preprocess the input so rest of the finctions work

maybe better to add this line directly in the GPSAnalytics().get_metrics method ?

In [ ]:
act.cluster_size = act.cluster_size.astype(int) #make sure we have the right format

In [ ]:
#I need to get the MAIN home location ID first, as in the data above one user_id may have different home location for each day.

# Group by 'user_id' and the date part of 'started_at'
grouped = act.groupby(['user_id', act['started_at'].dt.date])

# Find the most recurrent 'home_location_id' for each user and day
most_recurrent_home = grouped['home_location_id'].agg(lambda x: x.value_counts().idxmax())

# Most recurrent home_id per user
most_recurrent_home_id = most_recurrent_home.value_counts().idxmax()

# Create a mapping of user_id and date to the most recurrent home_location_id
mapping = most_recurrent_home.reset_index().set_index(['user_id','started_at']).to_dict()['home_location_id']

# Map the values to the original DataFrame to create the new column
act['main_home_location_id'] = act.set_index(['user_id', act['started_at'].dt.date]).index.map(mapping)

In [ ]:
act.head()

### Part 1
Process the action space metrics per user_id (all days, one_user) or per user_id_day (one day, one user)
- `GSPtoActionspace().compute_action_space(act, aggreg_method = 'user_id'/'user_id_day',plot_ellipses = False)`

In [ ]:
aggregation_level = 'user_id_day'  # Change this to 'user_id' or 'user_id_day'

In [ ]:
action_space = []
cols = ['user_id', 'lon', 'lat', 'cluster_size', 'cluster_info', 'location_id', 'home_location_id', 'num_trip', 'main_home_location_id', 'dist_from_home', 'imputed_purpose']


for user_id_ in act[aggregation_level].unique():
    
    act_ = act.loc[act[aggregation_level] == user_id_, cols].copy()
    # get the main home id at user_id scale
    main_home_id = act_.home_location_id.value_counts().idxmax()
    
    if main_home_id == 'not_detected':
        continue
    #elif np.any(act_.dist_from_home > 3000000):
    #    continue
    elif act_.main_home_location_id.unique() != act_.home_location_id.unique():
        continue
    elif np.all(act_.num_trip == 0):
        continue
    else:
        act_.drop(columns=['num_trip', 'dist_from_home'], inplace=True)
        
        # Assuming your DataFrame is called df
        act_ = gpd.GeoDataFrame(act_, geometry=gpd.points_from_xy(act_['lon'], act_['lat']), crs="EPSG:4326")
        
        # Change the CRS to EPSG:2056
        act_ = act_.to_crs("EPSG:2056")  # This should be a parameter to change in the method, but default set to this one
        
        # Now, you can access the transformed x and y coordinates
        act_['x'], act_['y'] = act_.geometry.x, act_.geometry.y
        
        # And drop duplicated locations (information of visit count is in cluster_size)
        act_ = act_.drop_duplicates(ignore_index=True)
        
        # Differentiate most frequent locations from all locations
        act_freq = act_.loc[act_.cluster_info.isin(['Most visited', 'Frequent visit'])].reset_index(drop=True)
        
        if len(act_) == 0:
            continue
        else:
        
            # Find mean center of a point array
            mc_ = mean_center(act_[['x', 'y']])
            # Find weighted mean center of a marked point pattern.
            wmc_ = weighted_mean_center(act_[['x', 'y']], act_.cluster_size)
            
            # Calculate Args of standard deviational ellipse for a point pattern
            if len(act_) > 2:
                sx, sy, theta = pointpats.centrography.ellipse(act_[['x', 'y']])
                theta_degree = np.degrees(theta)  # need degree of rotation to plot the ellipse
                # Calculate the surface of the SD ellipse of the action space, in square meters
                surface_ellipse = np.pi * sx * sy
            else:
                surface_ellipse = 0
            # sx, sy, theta, theta_degree
            
            # Calculate the ellipse motplotlib objet
            e = Ellipse(xy=wmc_, width=sx*2, height=sy*2, angle=-theta_degree, alpha=0.2, facecolor='#c9cfbc') #angle is rotation in degrees (anti-clockwise)
            
            # Find the main home coordinates
            find_home_geom = act_.loc[act_.location_id == act_.home_location_id.unique()[0], 'geometry']
            if len(find_home_geom) == 1:
                home_loc = np.array([find_home_geom.x, find_home_geom.y], dtype=float)
            elif len(find_home_geom) > 1:
                home_loc = np.array([find_home_geom[0].x, find_home_geom[0].y], dtype=float)
            
            # Sum of Euclidean distances between event points and a selected point.
            sum_dist_home = dtot(home_loc, act_[['x', 'y']].to_numpy())
            
            # Compute location regularity
            n_all = len(act_)
            n_freq = len(act_freq)
            regularity = n_freq / n_all
            
            # Calculate standard distance between all nodes and home
            std_dist_all = modified_std_distance(act_, home_loc)
            # Calculate standard distance between freq nodes and home
            if len(act_freq) > 0:
                std_dist_freq = modified_std_distance(act_freq, home_loc)
            else:
                std_dist_freq = np.nan
            
            # Calculate the proximity
            proximity = std_dist_freq / std_dist_all
            
            # Calculate the Euclidean median for a point pattern.
            median_ = euclidean_median(act_[['x', 'y']].to_numpy())[0]
            
            # Get area of convex hull for frequent (i.e. habitual) locations in square meters
            # And Coefficient of intensity
            if len(act_freq.location_id.unique()) > 2:
                freq_hull_area = MultiPoint(act_freq.geometry).convex_hull.area
                intensity = MultiPoint(act_freq.geometry).convex_hull.area / MultiPoint(act_.geometry).convex_hull.area
            elif len(act_freq.location_id.unique()) == 2:
                loc1 = act_freq[['x', 'y']].loc[act_freq.location_id == act_freq.location_id.unique()[0]].drop_duplicates()
                loc2 = act_freq[['x', 'y']].loc[act_freq.location_id == act_freq.location_id.unique()[1]].drop_duplicates()
            
                coord1 = loc1[['x', 'y']].to_numpy().flatten()
                coord2 = loc2[['x', 'y']].to_numpy().flatten()
            
                freq_hull_area = distance.euclidean(coord1, coord2)
                intensity = 0
            else:
                freq_hull_area = 0
                intensity = 0
            
            # Calculate euclidean distance between weighted mean center and main home location in meters
            home_shift = distance.euclidean(wmc_, home_loc.flatten())
            

            action_space_ = np.array(
                [user_id_, main_home_id, sum_dist_home, proximity, std_dist_all, std_dist_freq, median_, intensity,
                 regularity, n_all, n_freq, freq_hull_area, home_shift, e, surface_ellipse,wmc_[0],wmc_[1],sx,sy,theta_degree])
            action_space.append(action_space_)

columns_ = [aggregation_level, 'main_home_id', 'sum_dist_home', 'proximity', 'std_dist_all', 'std_dist_freq', 'median_', 'intensity', 'regularity', 'n_all', 'n_freq', 'freq_hull_area', 'home_shift', 'ellipse_2056', 'surface_ellipse','wmc_x','wmc_y','sx','sy','theta_degree']
AS = pd.DataFrame(action_space, columns=columns_)
AS.set_index(aggregation_level,inplace=True)

#Parse to float
col_2= ['sum_dist_home','proximity','std_dist_all', 'std_dist_freq','median_','intensity','regularity','n_all','n_freq','freq_hull_area','home_shift','surface_ellipse']
AS[col_2] = AS[col_2].astype(float)

In [ ]:
AS

### Part 2
Plot the ellipses, get one color per user and user_id on hover.

`MATTTEO` Please integrate this the previous methods in Past 1

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon, Point
import folium
import pyproj

# Create a list to store ellipse geometries
ellipse_geometries = []

# Define the source and target coordinate reference systems (CRS)
source_crs = "EPSG:2056"
target_crs = "EPSG:4326"

# Iterate through the DataFrame and add ellipses to the list
for idx, row in AS.reset_index().iterrows():
    # Extract ellipse information
    user_id = row[aggregation_level]
    ellipse = row['ellipse_2056']

    # Convert ellipse to Polygon geometry
    ellipse_polygon = Polygon(ellipse.get_verts())
    ellipse_coords = [[x,y] for x,y in ellipse_polygon.exterior.coords]

    ellipse_polygon = Polygon(ellipse.get_verts())
    ellipse_geometries.append(ellipse_polygon)

# Create a GeoDataFrame from the list of ellipse geometries
ellipse_gdf = gpd.GeoDataFrame(geometry=ellipse_geometries, crs=source_crs).to_crs(target_crs)

# Calculate the mean centroid of all ellipses
mean_lat = ellipse_gdf.geometry.apply(lambda geom: geom.centroid.x).mean()
mean_lon = ellipse_gdf.geometry.apply(lambda geom: geom.centroid.y).mean()

# Create a Folium map centered around the mean centroid
mymap = folium.Map(location=[mean_lon, mean_lat], zoom_start=12, control_scale=True)

# Iterate through the GeoDataFrame and add ellipses to the map
for idx, row in ellipse_gdf.iterrows():
    # Extract ellipse information
    user_id = idx  # Assuming user_id is the index in the GeoDataFrame
    ellipse_polygon = row['geometry']

    # Convert ellipse to coordinates for Folium
    ellipse_coords = list(ellipse_polygon.exterior.coords)
    ellipse_coords_t = [[y,x] for x,y in ellipse_coords]

    # Add the ellipse to the map
    folium.Polygon(locations=ellipse_coords_t, color='blue', fill=True, fill_color='blue', fill_opacity=0.02, popup=f"User ID: {user_id}").add_to(mymap)

# Display the map
mymap#.save("ellipse_map.html")


### Part 3
Covariance matrix of the Action Space Args we computed
- `GSPtoActionspace().covariance_matric(AS)`

In [ ]:
cov_corr_heatmap(AS[['sum_dist_home','proximity','std_dist_all', 'std_dist_freq','median_','intensity','regularity','n_all','n_freq','freq_hull_area','home_shift','surface_ellipse']], 
                 method='corr',cmap = sns.diverging_palette(360,65, l=80, as_cmap=True))

### Part 4
More plots, including the points from `act` and ellipse from `AS`
- `GSPtoActionspace().plot_action_space(act, AS, user_subset = ['CH15029', 'CH16871'], how = 'vignette'/'folium', save = False)` 

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import contextily as ctx

# Create a GeoDataFrame for the points from act_
print(act_.head())
geometry = [Point(x, y) for x, y in zip(act_['x'], act_['y'])]
gdf = gpd.GeoDataFrame(act_, geometry=geometry, crs='EPSG:2056')

# Create a Matplotlib figure
fig, ax = plt.subplots(figsize=(10, 10))


# Define the source and target coordinate reference systems (CRS)
#source_crs = "EPSG:2056"
#target_crs = "EPSG:4326"
## Create a PyProj transformer for the conversion
#transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)

# Plot Ellipse
e = Ellipse(xy=wmc_, width=sx*2, height=sy*2, angle=-theta_degree, alpha=0.2, facecolor='#c9cfbc') #angle is rotation in degrees (anti-clockwise)
ellipse_polygon = Polygon(e.get_verts())
ellipse_coords = np.array(ellipse_polygon.exterior.coords.xy)
#ellipse_coords_4326 = np.column_stack(transformer.transform(x, y) for x, y in zip(ellipse_coords[0], ellipse_coords[1]))
ax.fill(ellipse_coords[0], ellipse_coords[1], color='red', alpha=0.2, edgecolor='red')

# Plot points from act_
ax.scatter(gdf['x'], gdf['y'], color='red', marker='o', s=50, label='Points from act_')

# Plot wmc_ in green
ax.scatter(wmc_[0], wmc_[1], color='green', marker='o', s=50, label='Weighted mean center')

# Plot home_loc from act_ with a customized icon
ax.scatter(home_loc[0], home_loc[1], color='black', marker='^', s=100, label='Home Location')

# Add legend
ax.legend()

# Set the extent of the plot based on the bounding box of the GeoDataFrame
minx, miny, maxx, maxy = gdf.total_bounds
ax.set_xlim(minx * 0.998, maxx * 1.002)
ax.set_ylim(miny * 0.998, maxy * 1.002)

# Plot basemap using OpenStreetMap
ctx.add_basemap(ax, crs='EPSG:2056', source=ctx.providers.OpenStreetMap.Mapnik)

# Show the plot
plt.show()


In [ ]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium
from folium.plugins import MarkerCluster, Draw
import numpy as np

# Convert wmc_ to 'EPSG:4326'
wmc_4326 = gpd.GeoSeries(Point(wmc_[0], wmc_[1]), crs='EPSG:2056').to_crs("EPSG:4326").iloc[0]
wmc_4326 = [wmc_4326.y, wmc_4326.x]  # Flip lat and lon to match Folium format

# Create a GeoDataFrame for the points from act_
geometry = [Point(lon, lat) for lon, lat in zip(act_['lon'], act_['lat'])]
gdf = gpd.GeoDataFrame(act_, geometry=geometry, crs='EPSG:4326')

# Create a GeoDataFrame for the home_loc
home_loc_geometry = [Point(home_loc[0], home_loc[1])]
home_loc_gdf = gpd.GeoDataFrame(geometry=home_loc_geometry, crs='EPSG:2056').to_crs("EPSG:4326")

# Create a GeoDataFrame for the ellipse
# Define the source and target coordinate reference systems (CRS)
source_crs = "EPSG:2056"
target_crs = "EPSG:4326"
# Create a PyProj transformer for the conversion
transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)

# Create Ellipse
e = Ellipse(xy=wmc_, width=sx*2, height=sy*2, angle=-theta_degree, alpha=0.2, facecolor='#c9cfbc') #angle is rotation in degrees (anti-clockwise)
ellipse_polygon = Polygon(e.get_verts())
ellipse_coords = [[x,y] for x,y in ellipse_polygon.exterior.coords]
# Convert coordinates to EPSG:4326
ellipse_coords_4326 = [transformer.transform(x, y) for x, y in ellipse_coords]
ellipse_coords_4326_t = [[y,x] for x,y in ellipse_coords_4326]
ellipse_gdf = gpd.GeoDataFrame(geometry=[Polygon(ellipse_coords_4326)], crs='EPSG:4326')

# Create a Folium map centered around the mean coordinates
mean_lat, mean_lon = gdf['lat'].mean(), gdf['lon'].mean()
map_center = [mean_lat, mean_lon]
mymap = folium.Map(location=map_center, zoom_start=12, control_scale=True)

# Add basemap using OpenStreetMap
folium.TileLayer('openstreetmap').add_to(mymap)

# Add Ellipse
#folium.Polygon(locations=ellipse.exterior.coords, color='#c9cfbc', fill=True, fill_color='#c9cfbc', fill_opacity=0.2).add_to(mymap)
folium.GeoJson(ellipse_gdf, name='polygon',zoom_on_click=True, style_function=lambda x: {'color': 'red'}).add_to(mymap)

# Add points from act_ with CircleMarkers
for idx, row in gdf.iterrows():
    folium.CircleMarker(location=[row['lat'], row['lon']], radius=5, color='red', fill=True, fill_color='red', fill_opacity=0.7, popup=f"{row['user_id']}").add_to(mymap)

# Add wmc_ in green
folium.CircleMarker(location=wmc_4326, radius=5, color='green', fill=True, fill_color='green', fill_opacity=1, popup="Weighted mean ceter").add_to(mymap)

# Add home_loc from act_ with a customized icon
folium.Marker(location=[home_loc_gdf.geometry.y.iloc[0], home_loc_gdf.geometry.x.iloc[0]], icon=folium.Icon(color='red', icon='home')).add_to(mymap)

# Display the map
mymap


### Part 5
**Method**
- `GPStoActionspace().inno_rate(mtf_, AS_day, user_id_, phase=None, treatment=None)`

**Objective**

The objective here is to plot the innovation rate

**The necessary data for this method are :**
- `GPStoGraph().get_graphs()` -> mtf_ 
- `GSPtoActionspace().compute_action_space(act, aggreg_method = 'user_id_day')` -> AS_day

**phase and treatment**

GPS data often come with a treatment e.g. {control, treat_1, treat_2} or phase column e.g. {before, after}. Here it is not the case, so no hue is necessary on the plots. But please keep the option of plotting hue for different treatment or phase

In [ ]:
mtf_treatment = pd.merge(mtf_[['DiGraph_motif','motif_flat']], AS_day.set_index('user_id_day'), left_index=True, right_index=True, how='left')
mtf_treatment.head(2)

In [ ]:
def get_inno_rate_per_phase(mtf_useridday, user_id_, col_digraph_motif = 'DiGraph_motif', phase=None, treatment=None):
    '''
    Compute the innovation rate from motif
    user_id: list of users to compute the innovation rate
    mtf_useridday: df of DiGraph_motifs with user_id_days in index'''

    #df_innov_rate = pd.DataFrame(index=range(100),columns=user_id)

    #isolate rows for phase X only
    if phase != None:
        mtf_useridday = mtf_useridday[mtf_useridday.phase == phase]
    #isolate rows for treatment X only
    if treatment != None:
        mtf_useridday = mtf_useridday[mtf_useridday.treatment == treatment]
    #resample for continuous days
    graphs = mtf_useridday[mtf_useridday.user_id == user_id_].set_index('date').resample('D').ffill()
    #read the first graph G0 for user_id
    G0 = graphs[col_digraph_motif][0]
    #init inno rate list
    innovation_rate = []
    #then compute the next graph and compare with G0
    for G in graphs[col_digraph_motif][1:]:
        innovation_rate.append(nx.compose(G0,G).number_of_nodes())
        G0 = nx.compose(G0,G)
    #x = np.arange(0, len(innovation_rate), 1).tolist()
    #y = innovation_rate
    
    return innovation_rate

the script below is implemented in case of phase and treatment. In the sample data I have no phase and no treatment, but I still want to be able to plot the innovation rate. Sorry it is messy. It you cannot adapt I will do it myself later on.

In [ ]:
to_plot = 800
#Init df to be populated with the innovation rates per cluster
df_innov_rate_1 = pd.DataFrame(index=range(500),columns=user_id_clstr1[:to_plot]) 
df_innov_rate_2 = pd.DataFrame(index=range(500),columns=user_id_clstr2[:to_plot]) 
df_innov_rate_3 = pd.DataFrame(index=range(500),columns=user_id_clstr3[:to_plot]) 


treatment_ = 'Pricing' #Pricing, Nudging, Control


#Init df to have the mean innovation rates in a single df
mean_innov_rate = pd.DataFrame(index=range(500),columns=['exclusive_phase1', 'moderate_phase1', 'mixed_phase1','exclusive_phase2', 'moderate_phase2', 'mixed_phase2'])

for phase_ in [1,2]:
    #Exclusive car users, user_id_clstr1
    for user_id_ in user_id_clstr1[:to_plot]:
        try:
            y = get_inno_rate_per_phase(mtf_treatment, user_id_, phase=phase_, treatment=treatment_)
            x = np.arange(0, len(y), 1).tolist()
            df_innov_rate_1.loc[x, user_id_] = y
        except:
            continue        
    
    mean_innov_rate.loc[:, 'exclusive_phase%d' %phase_] = df_innov_rate_1.mean(axis=1)
    
    #Moderate car users, user_id_clstr2
    for user_id_ in user_id_clstr2[:to_plot]:
        try:
            y = get_inno_rate_per_phase(mtf_treatment, user_id_, phase=phase_, treatment=treatment_)
            x = np.arange(0, len(y), 1).tolist()
            df_innov_rate_2.loc[x, user_id_] = y
        except:
            continue        
    
    mean_innov_rate.loc[:, 'moderate_phase%d' %phase_] = df_innov_rate_2.mean(axis=1)

    #Mixed car users, user_id_clstr3
    for user_id_ in user_id_clstr3[:to_plot]:
        try:
            y = get_inno_rate_per_phase(mtf_treatment, user_id_, phase=phase_, treatment=treatment_)
            x = np.arange(0, len(y), 1).tolist()
            df_innov_rate_3.loc[x, user_id_] = y
        except:
            continue        
    
    mean_innov_rate.loc[:, 'mixed_phase%d' %phase_] = df_innov_rate_3.mean(axis=1)

mean_innov_rate.head(3)    

In [ ]:
plot_innov_rate_cluster1 = sns.lineplot(data=mean_innov_rate[['exclusive_rate_phase1','moderate_rate_phase1', 'mixed_rate_phase1']][:25]) #, legend=False
#plot_innov_rate_cluster1.get_figure().savefig("innov_rate_modal_clus__phase1%s.png"%treatment_, dpi=300)